In [1]:
pip install panda


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyarrow


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pyarrow.parquet as pq
import pandas as pd

table = pq.read_table('timediaries.parquet')
df = table.to_pandas()
df.size
# df['A1_What are you doing?'].unique()


1310543

In [7]:
# generating csv for activity stream

table = pq.read_table('timediaries.parquet')
df = table.to_pandas()
df.size

df['activity'] = df['A1_What are you doing?']
df['timestamp'] = pd.to_datetime(df['answertimestamp'])

df = df.sort_values(by=['userid', 'timestamp'])

df['activity_change'] = (df['activity'] != df['activity'].shift(1)) | (df['userid'] != df['userid'].shift(1))
df['activity_block'] = df['activity_change'].cumsum()

df['next_timestamp'] = df.groupby('userid')['timestamp'].shift(-1)  # Get the next timestamp within each user group
df['duration_seconds'] = (df['next_timestamp'] - df['timestamp']).dt.total_seconds().fillna(0)


grouped = df.groupby(['userid', 'activity', 'activity_block']).agg({'duration_seconds': 'sum'}).reset_index()

grouped['minutes'] = (grouped['duration_seconds'] // 60).astype(int) 
grouped['seconds'] = (grouped['duration_seconds'] % 60).astype(int)

grouped['duration'] = grouped.apply(lambda row: f"{row['minutes']} min {row['seconds']} sec", axis=1)

merged_df = pd.merge(df[['userid', 'timestamp', 'activity', 'activity_block']], 
                     grouped[['userid', 'activity_block', 'duration']], 
                     on=['userid', 'activity_block'], 
                     how='left')

merged_df = merged_df.drop_duplicates(subset=['userid', 'activity_block'])

merged_df[['userid', 'timestamp', 'activity', 'duration']].to_csv('activity_stream.csv', index=False)

print("CSV file created.")

activity_df = pd.read_csv('activity_stream.csv')

print(activity_df.head())

CSV file created.
   userid                timestamp                               activity  \
0       0  2018-05-09 19:33:48.395         Guardo Youtube, Serie-Tv, ecc.   
1       0  2018-05-09 21:09:51.600  Cinema, Teatro, Concerto, Mostra, ...   
2       0  2018-05-09 21:32:45.845         Guardo Youtube, Serie-Tv, ecc.   
3       0  2018-05-09 23:33:07.865         Leggo un libro; ascolto musica   
4       0  2018-05-10 00:02:20.655                                Dormire   

         duration  
0    96 min 3 sec  
1   22 min 54 sec  
2  120 min 22 sec  
3   29 min 12 sec  
4  755 min 54 sec  


In [ ]:
# maps emojis to numerics

table = pq.read_table('timediaries.parquet')
dataset = table.to_pandas().to_csv('dataset.csv', index=False)

df = pd.read_csv('dataset.csv')
df.head()

unique_emojis = df['A6_What is your mood?'].unique()
print("Unique emojis in the dataset:", unique_emojis)
emoji_meaning_mapping = {
    '😞': 0,  # sad
    '🙁': 1,  # down
    '😐': 2,  # neutral
    '😊': 3,  # happy
    '😁': 4,  # excited
    # '<NA>': 5,  # unanswered
}
emoji_text_mapping = {
    '😞': 'sad',
    '🙁': 'down', 
    '😐': 'neutral',
    '😊': 'happy', 
    '😁': 'excited',
    # '<NA>': 'unanswered' 
}

df['mood'] = df['A6_What is your mood?'].map(emoji_meaning_mapping)
df['mood'] = df['mood'].fillna(5).astype(int)

unmapped_moods = df[df['mood'].isna()]['A6_What is your mood?'].unique()
if len(unmapped_moods) > 0:
    print("Unmapped moods (emojis):", unmapped_moods)
# df.head()


df['mood_text'] = df['A6_What is your mood?'].map(emoji_text_mapping)
df['mood_text'] = df['mood_text'].fillna('unanswered')

unmapped_moods = df[df['mood_text'].isna()]['A6_What is your mood?'].unique()
if len(unmapped_moods) > 0:
    print("Unmapped moods (emojis):", unmapped_moods)

# df.tail()

Unique emojis in the dataset: ['😊' '😁' nan '😐' '😞' '🙁']


,experimentid,userid,questiontimestamp,answerduration,answertimestamp,delta,notificationtimestamp,questionid,A1_What are you doing?,A2_How are you moving?,A3_Where are you?,A5_With whom are you?,A6_What is your mood?,mood,mood_text
100806,smartUnitnII,157,2018-06-07 09:51:21.624,4007,2018-06-07 10:14:25.390,1384173,2018-06-07 09:51:21.217,d0d78160ab3d87c7c6949e90a5897d7f49fa2486,Studio,Biblioteca UNITN,NaN,Da solo,😊,3,happy
100807,smartUnitnII,157,2018-06-07 13:51:21.534,4434,2018-06-07 13:51:37.486,16216,2018-06-07 13:51:21.270,d0d78160ab3d87c7c6949e90a5897d7f49fa2486,Studio,Biblioteca UNITN,NaN,Da solo,😊,3,happy
100808,smartUnitnII,157,2018-06-07 15:51:21.530,5875,2018-06-07 16:27:00.180,2139027,2018-06-07 15:51:21.153,d0d78160ab3d87c7c6949e90a5897d7f49fa2486,Studio,Biblioteca UNITN,NaN,Partner,😊,3,happy
100809,smartUnitnII,157,2018-06-07 17:51:21.519,5217,2018-06-07 17:51:53.257,12413,2018-06-07 17:51:40.844,d0d78160ab3d87c7c6949e90a5897d7f49fa2486,Studio,Biblioteca UNITN,NaN,Partner,😊,3,happy
100810,smartUnitnII,157,2018-06-07 21:51:21.516,5036,2018-06-07 22:06:50.274,929148,2018-06-07 21:51:21.126,d0d78160ab3d87c7c6949e90a5897d7f49fa2486,Studio,"Casa, Appartamento, Stanza",NaN,Da solo,😊,3,happy


In [9]:
# generating csv for mood stream

df['mood_emj'] = df['A6_What is your mood?']
df['timestamp'] = pd.to_datetime(df['answertimestamp'])

df = df.sort_values(by=['userid', 'timestamp'])

df['mood_change'] = (df['mood'] != df['mood'].shift(1)) | (df['userid'] != df['userid'].shift(1))
df['mood_block'] = df['mood_change'].cumsum()

df['next_timestamp'] = df.groupby('userid')['timestamp'].shift(-1)  # Get the next timestamp within each user group
df['duration_seconds'] = (df['next_timestamp'] - df['timestamp']).dt.total_seconds().fillna(0)


grouped = df.groupby(['userid', 'mood', 'mood_block']).agg({'duration_seconds': 'sum'}).reset_index()

grouped['minutes'] = (grouped['duration_seconds'] // 60).astype(int) 
grouped['seconds'] = (grouped['duration_seconds'] % 60).astype(int)

grouped['duration'] = grouped.apply(lambda row: f"{row['minutes']} min {row['seconds']} sec", axis=1)

merged_df = pd.merge(df[['userid', 'timestamp', 'mood', 'mood_emj', 'mood_text', 'mood_block']], 
                     grouped[['userid', 'mood_block', 'duration']], 
                     on=['userid', 'mood_block'], 
                     how='left')

merged_df = merged_df.drop_duplicates(subset=['userid', 'mood_block'])

merged_df[['userid', 'timestamp', 'mood', 'mood_text', 'duration']].to_csv('mood_stream.csv', index=False)

print("CSV file created.")

mood_df = pd.read_csv('mood_stream.csv')

mood_df.size
print(mood_df.head())

CSV file created.
   userid                timestamp  mood   mood_text            duration
0       0  2018-05-09 19:33:48.395     3       happy  180.0 min 25.0 sec
1       0  2018-05-09 22:34:13.508     4     excited    88.0 min 7.0 sec
2       0  2018-05-10 00:02:20.655     5  unanswered  755.0 min 54.0 sec
3       0  2018-05-10 12:38:14.671     3       happy   73.0 min 13.0 sec
4       0  2018-05-10 13:51:27.778     4     excited   12.0 min 41.0 sec


In [ ]:

def convert_value(value):
    
    try:
        if '.' in value:
            return float(value)
        elif len(value) > 10:
                return value
        else:
            return int(value)
    except ValueError:
        return value 

def csv_to_json(csv_file, json_file):
    
    with open(csv_file, mode='r') as f:
        reader = csv.DictReader(f)
        rows = []
        for row in reader:
            converted_row = {key: convert_value(value) for key, value in row.items()}
            rows.append(converted_row)

    with open(json_file, mode='w') as f:
        json.dump(rows, f, indent=4)

csv_file = "../auxilary datas/activity_stream.csv"
json_file = "../auxilary datas/activity_stream.json"
csv_to_json(csv_file, json_file)
csv_file = "../auxilary datas/mood_stream.csv"
json_file = "../auxilary datas/mood_stream.json"
csv_to_json(csv_file, json_file)